In [1]:
!pip install  accelerate peft bitsandbytes transformers trl triton

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 

In [2]:
#CMAKE_ARGS="-DLLAMA_CUDA=on" pip install llama-cpp-python #gpu
!pip install llama-cpp-python #cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 MB 9.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.7 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.2-cp310-cp310-linux_x86_64.whl size=3410018 sha256=20ec6fc3c951aad90640d045054129244f9bfb98215d3cc3d837a40d2d87a3b8
  Stored in directory: /root/.cache/pip/wheels/49/1c/f0/6c1ed6032d5827dea28df8a8df860d90ca1bcd7b3fa455f5e1
Successfully built llama-cpp-python


In [3]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [4]:
import pandas as pd

# Read the XLSX file
xlsx_file = pd.read_csv("/content/output_malayalam.csv")

In [5]:
# @title prepare data

input_prompt = """Below is a Human Input, write appropriate Response based on the input.

### Input:
{}

### Response:
{}"""


In [6]:
# The model that you want to train from the Hugging Face hub
#model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
model_name = "unsloth/Llama-3.2-1B-Instruct" #pretrained model
#model_mame = "/content/final_weights_new"

# The instruction dataset to use
#dataset_name = "mlabonne/guanaco-llama2-1k"

# Fine-tuned model name
new_model = "llama3.2-fine-tuned"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 20

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 1

# Batch size per GPU for evaluation
per_device_eval_batch_size = 1

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4 #0.0002 2x10-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0} # "auto"

In [7]:
# Load dataset (you can process it here)
#dataset = load_dataset(dataset_name, split="train")
%cd "/content"
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)
print(compute_dtype)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training
EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    inputs       = examples["question"]
    outputs      = examples["answer"]
    texts = []
    for input, output in zip(inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = input_prompt.format(input, output) + EOS_TOKEN
        texts.append(text)
    print(texts)
    return { "text" : texts, }
pass
'''
def formatting_prompts_func(examples):
    inputs       = examples["instruction"]
    outputs      = examples["output"]
    texts = []
    for input, output in zip(inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = input_prompt.format(input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass'''

from datasets import load_dataset
dataset = load_dataset('csv', data_files='/content/output_malayalam.csv',split="train")
#dataset = load_dataset("nmdr/Mini-Physics-Instruct-1k", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)
# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
      target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard",

)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,

)

# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained(new_model)

/content
torch.float16


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/927 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

...

Map:   0%|          | 0/497 [00:00<?, ? examples/s]

['Below is a Human Input, write appropriate Response based on the input.\n\n### Input:\nവിളകൾ എന്താണ്?\n\n### Response:\nവിളകൾ കൃഷിക്കാർ വളർത്തുന്ന സസ്യങ്ങളാണ്. ഇന്ത്യൻ സമ്പദ് വ്യവസ്ഥയിൽ കൃഷി പ്രധാനപ്പെട്ട പങ്ക് വഹിക്കുന്നു.<|eot_id|>', 'Below is a Human Input, write appropriate Response based on the input.\n\n### Input:\nഖരിഫ് വിളകൾ എന്താണ്?\n\n### Response:\nമൺസൂൺ സീസണിൽ വളർത്തുന്ന വിളകളെയാണ് ഖരിഫ് വിളകൾ എന്ന് പറയുന്നത്. ഉദാഹരണം: ചോളം, മില്ലറ്റ്, കോട്ടൺ.<|eot_id|>', 'Below is a Human Input, write appropriate Response based on the input.\n\n### Input:\nറാബി വിളകൾ എന്താണ്?\n\n### Response:\nശൈത്യകാലത്ത് വളർത്തുകയും വസന്തകാലത്ത് വിളവെടുക്കുകയും ചെയ്യുന്ന വിളകളെയാണ് റാബി വിളകൾ എന്ന് പറയുന്നത്. ഉദാഹരണം: ഗോതമ്പ്, ഗ്രാം, കടുക്.<|eot_id|>', 'Below is a Human Input, write appropriate Response based on the input.\n\n### Input:\nസൈഡ് ക്രോപ്പുകൾ എന്താണ്?\n\n### Response:\nഖാരിഫ്, റാബി സീസണുകൾക്കിടയിൽ വളർത്തുന്ന വിളകളാണ് സൈഡ് ക്രോപ്പുകൾ. ഉദാഹരണം: കുക്കുമ്പർ, മത്തങ്ങ, കയ്പേറിയ പൊറോട്ട, തണ്ണിമത്തൻ.

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:309: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/497 [00:00<?, ? examples/s]

Step,Training Loss
25,1.153800
50,1.101200
75,0.773000
100,0.674900
125,0.710000
150,0.630700
175,0.627300
200,0.620400
225,0.628100
250,0.526000


In [8]:
!rm -rf "final_weights_new"

In [9]:
##Inference
inputs = tokenizer(
[
    input_prompt.format(
        "ജനിതക നവീകരണത്തിനുള്ള ഉപകരണങ്ങൾ എന്തൊക്കെയാണ്?", # input
        "",   # leave blank as response generated by AI

    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 512, use_cache = True)
generated_text = tokenizer.batch_decode(outputs)[0]
first_response = generated_text.split('### Response:')[1].strip()
output = first_response.split('###')[0].strip()
print("the response is: ",output)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


the response is:  ഹൈബ്രിഡ് ബ്രീഡിംഗ്, ട്രാൻസ്‌ജീനിക് സാങ്കേതികവിദ്യ എന്നിവയാണ് പ്രധാന ഉപകരണങ്ങൾ. ഉയർന്ന വിളവ്, വരൾച്ച പ്രതിരോധം, പ്രാണി-രോഗസഹിഷ്ണുത എന്നിവ മുഖേന പ്രതിപാദിക്കുന്നു. വിളകൾ പരിവർത്തനപ്പെടുത്താൻ സംരംഭിക്കുന്ന വിളകൾ ഒരുക്കിയിട്ടുണ്ട്. ഇത് നാടന്‍ കൃഷിയുടെ ഉല്പാദനക്ഷമത നിലനിർത്താൻ സഹായിക്കുന്നു. വിളകൾ കൃഷിക്കായുള്ള സാങ്�


In [10]:
# Reload model in FP16 and merge it with LoRA weights w = w+del(w)
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload() #W=w+del(w)

# Reload tokenizer to save it
#tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
#tokenizer.pad_token = tokenizer.eos_token
#tokenizer.padding_side = "right"

In [11]:
output_dir = "final_weights_new"
model.save_pretrained(output_dir)

In [12]:

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
tokenizer.save_pretrained(output_dir)

('final_weights_new/tokenizer_config.json',
 'final_weights_new/special_tokens_map.json',
 'final_weights_new/tokenizer.json')

In [13]:
import shutil

# Path to the directory
folder_path = "/content/final_weights_new"
output_zip = "/content/final_weights_new.zip"

# Compress the folder into a zip file
shutil.make_archive(folder_path, 'zip', folder_path)

# Provide a link to download the zip file
from google.colab import files
files.download(output_zip)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [14]:
# Run text generation pipeline with our next model
# Load model directly </s>
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("/content/final_weights_new")
model = AutoModelForCausalLM.from_pretrained("/content/final_weights_new", device_map = "auto")
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=2048)
prompt=input_prompt.format(
        "PMKSY പദ്ധതിയുടെ പ്രതീക്ഷകൾ എന്തെല്ലാമാണ്??", # input
        "", # leave blank as response generated by AI

    )
result = pipe(prompt, temperature=0.05)
generated_text  = result[0]['generated_text']

first_response = generated_text.split('### Response:')[1].strip()
first_response = first_response.split("\n")[0]

print(first_response)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


PMKSY പദ്ധതിയുടെ പ്രതീക്ഷകളിൽ 5,30,500 തൊഴിൽ അവസരങ്ങൾ സൃഷ്ടിക്കുക, കർഷകർക്ക് കൂടുതൽ വില ലഭ്യമാക്കുക, കൃഷി ഉത്പന്നങ്ങളുടെ മിതമായ വാടകങ്ങൾ കുറക്കുക, കർഷകന്റെ കാര്യഭാരത്തിന്റെ വില്യഭരണം മെച്ചപ്പെടുത്തുക. ചെലവ് കുറവുള്ള തൊഴിൽ അവസരങ്ങൾ സൃഷ്ടിക്കുക, കർഷകന്റെ കാര്യഭാരത്തിന്റെ വില്യഭരണം മെച്ചപ്പെടുത്തുക. കൂടാതെ പച്ചപ്പുല്ല് കൃഷിയിലേക്ക് എത്തിക്കുന്ന വിലയും വില്യഭരണം ഉറപ്പിക്കുക. ഇതിനെ പ്രതിരോധിക്കാൻ കൃഷി ഉത്പന്നങ്ങളുടെ പരിപാലനത്തിനും വില്യഭരണം മെച്ചപ്പെടുത്തലിനും സഹായിക്കുന്നു. ചെലവ് കുറവുള്ള തൊഴിൽ അവസരങ്ങൾ സൃഷ്ടിക്കുക, കർഷകന്റെ കാര്യഭാരത്തിന്റെ വില്യഭരണം മെച്ചപ്പെടുത്തുക, കൂടാതെ പച്ചപ്പുല്ല് കൃഷിയിലേക്ക് എത്തിക്കുന്ന വിലയും വില്യഭരണം ഉറപ്പിക്കുക. ഇതിനെ പ്രതിരോധിക്കാൻ കൃഷി ഉത്പന്നങ്ങളുടെ പരിപാലനത്തിനും വില്യഭരണം മെച്ചപ്പെടുത്തലിനും സഹായിക്കുന്നു. ചെലവ് കുറവുള്ള തൊഴിൽ അവസരങ്ങൾ സൃഷ്ടിക്കുക, കർഷകന്റെ കാര്യഭാരത്തിന്റെ വില്യഭരണം മെച്ചപ്പെടുത്തുക, കൂടാതെ പച്ചപ്പുല്ല് കൃഷിയിലേക്ക് എത്തിക്കുന്ന വിലയും വില്യഭരണം ഉറപ്പിക്കുക. ഇതിനെ പ്രതിരോധിക്കാൻ കൃഷി ഉത്പന്നങ്ങളുടെ പരിപാലനത്തിനും വില്യഭരണം മെച്ചപ്പെടുത

In [15]:
prompt=input_prompt.format(
        "ചിപ്പിക്കൂണ്‍ കൃഷിയിൽ സ്‌പോ വളര്‍ച്ച എത്ര ദിവസം കൊണ്ട് കാണാമെന്നു?", # input
        "", # leave blank as response generated by AI

    )
result = pipe(prompt, temperature=0.05)
generated_text  = result[0]['generated_text']

first_response = generated_text.split('### Response:')[1].strip()
first_response = first_response.split("\n")[0]

print(first_response)

സ്‌പോ വളര്‍ച്ച 10-15 ദിവസത്തിനകം വളരുന്നതിനുള്ളായിരിക്കും. അതിനാൽ, 20-25 ദിവസത്തിനകം കാണാമെന്നു പ്രതിരോധിക്കാം. അതിനാൽ, 30-35 ദിവസത്തിനകം കാണാമെന്നു പ്രതിരോധിക്കാം. അതിനാൽ, ചിപ്പിക്കൂണ്‍ കൃഷിയിലെ സ്‌പോ വളര്‍ച്ചയുടെ അളവ് എങ്ങനെ വളരുന്നു?


In [16]:
prompt=input_prompt.format(
        "400 കിലോ എരുമയ്ക്ക് എത്ര തീറ്റയും വൈക്കോലും കൊടുക്കണം?", # input
        "", # leave blank as response generated by AI

    )
result = pipe(prompt, temperature=0.05)
generated_text  = result[0]['generated_text']

first_response = generated_text.split('### Response:')[1].strip()
first_response = first_response.split("\n")[0]

print(first_response)

400 കിലോ ശരീരഭാരം ഉള്ള എരുമയ്ക്ക് 2 കിലോ തീറ്റയും 8-10 കിലോ വൈക്കോലും 3-5 കിലോ പച്ചപ്പുല്ലും നൽകണം. ഇത് ശരീരഭാരം കുറയ്ക്കും, വാതാശയത്തെ കുറയ്ക്കും, പച്ചപ്പുല്ല് കലക്കാം. വേഗത്തിൽ നിന്ന് എടുക്കണം. അതായത്, എരുമയ്ക്ക് 1 കിലോ തീറ്റയും 2-3 കിലോ വൈക്കോലും 3-5 കിലോ പച്ചപ്പുല്ലും നൽകണം. ഇത് ശരീരഭാരം കുറയ്ക്കാനും, വാതാശയത്തെ കുറയ്ക്കാനും, പച്ചപ്പുല്ല് കലക്കാനും നല്ലതാണ്. വേഗത്തിൽ നിന്ന് എടുക്കണം. അതായത്, എരുമയ്ക്ക് 1 കിലോ തീറ്റയും 2-3 കിലോ വൈക്കോലും 3-5 കിലോ പച്ചപ്പുല്ലും നൽകണം. ഇത് ശരീരഭാരം കുറയ്ക്കാനും, വാതാശയത്തെ കുറയ്ക്കാനും, പച്ചപ്പുല്ല് കലക്കാനും നല്ലതാണ്. വേഗത്തിൽ നിന്ന് എടുക്കണം. അതായത്, എരുമയ്ക്ക് 1 കിലോ തീറ്റയും 2-3 കിലോ വൈക്കോലും 3-5 കിലോ പച്ചപ്പുല്ലും നൽകണം. ഇത് ശരീരഭാരം കുറയ്ക്കാനും, വാതാശയത്തെ കുറയ്ക്കാനും, പച്ചപ്പുല്ല് കലക്കാനും നല്ലതാണ്. വേഗത്തിൽ നിന്ന് എടുക്കണം. അതായത്, എരുമയ്ക്ക് 1 കിലോ തീറ്റയും 2-3 കിലോ വൈക്കോലും 3-5 കിലോ പച്ചപ്പുല്ലും നൽകണം. ഇത് ശരീരഭാരം കുറയ്ക്കാനും, വാതാശയത്തെ കുറയ്ക്കാനും, പച്ചപ്പുല്ല് കലക്കാനും നല്ലതാണ്. വേഗത്തിൽ നിന്ന് എടുക്കണം. അതായത്, എരുമയ്ക്ക് 1 കില

In [18]:
prompt=input_prompt.format(
        "PMKSYയുടെ മുഖ്യ ലക്ഷ്യം എന്താണ്?", # input
        "", # leave blank as response generated by AI

    )
result = pipe(prompt, temperature=0.05)
generated_text  = result[0]['generated_text']

first_response = generated_text.split('### Response:')[1].strip()
first_response = first_response.split("\n")[0]

print(first_response)

PMKSY പദ്ധതിയുടെ മുഖ്യ ലക്ഷ്യം കൃഷി രംഗത്തെ വികസനം പ്രോത്സാഹിപ്പിച്ച് കാർഷിക ഉത്പന്നങ്ങളുടെ പ്രോസസ്സിംഗ് വഴി ക്ഷയവും പാഴ്‌വഴിയും കുറക്കുകയും ചെയ്യുകയുംാണ്. ഇത് കർഷകരുടെ കുറവും വിലയും പരിഹരിക്കുകയും ക്ഷയസ്രീവിലായിരിക്കുകയും ചെയ്യും. കൂടാതെ, പദ്ധതിയുടെ ലക്ഷ്യം കൃഷി രംഗത്തെ വികസനം പ്രോത്സാഹിപ്പിച്ച് കാർഷിക ഉത്പന്നങ്ങളുടെ പ്രോസസ്സിംഗ് വഴി ക്ഷയവും പാഴ്‌വഴിയും കുറക്കുകയും ചെയ്യുകയും ചെയ്യും. അതായത്, കർഷകരുടെ കുറവും വിലയും പരിഹരിക്കുകയും ക്ഷയസ്രീവിലായിരിക്കുകയും ചെയ്യും. കൂടാതെ, പദ്ധതിയുടെ ലക്ഷ്യം കൃഷി രംഗത്തെ വികസനം പ്രോത്സാഹിപ്പിച്ച് കാർഷിക ഉത്പന്നങ്ങളുടെ പ്രോസസ്സിംഗ് വഴി ക്ഷയവും പാഴ്‌വഴിയും കുറക്കുകയും ചെയ്യുകയും ചെയ്യും. അതായത്, കർഷകരുടെ കുറവും വിലയും പരിഹരിക്കുകയും ക്ഷയസ്രീവിലായിരിക്കുകയും ചെയ്യും. കൂടാതെ, പദ്ധതിയുടെ ലക്ഷ്യം കൃഷി രംഗത്തെ വികസനം പ്രോത്സാഹിപ്പിച്ച് കാർഷിക ഉത്പന്നങ്ങളുടെ പ്രോസസ്സിംഗ് വഴി ക്ഷയവും പാഴ്‌വഴിയും കുറക്കുകയും ചെയ്യുകയും ചെയ്യും. അതായത്, കർഷകരുടെ കുറവും വിലയും പരിഹരിക്കുകയും ക്ഷയസ്രീവിലായിരിക്കുകയും ചെയ്യും. കൂടാതെ, പദ്ധതിയുടെ ലക്ഷ്യം കൃഷി രംഗത്തെ വികസനം പ്രോത്സാഹി

In [19]:
!pip install vllm haystack-ai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.9/198.9 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.0/87.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 391.4/391.4 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.9/101.9 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 59

In [20]:
# we prepend "nohup" and postpend "&" to make the Colab cell run in background
! nohup python -m vllm.entrypoints.openai.api_server \
                  --model /content/final_weights_new \
                  --dtype auto \
                  --max-model-len 2048 \
                  > vllm.log &

nohup: redirecting stderr to stdout


In [21]:
# we check the logs until the server has been started correctly
!while ! grep -q "Application startup complete" vllm.log; do tail -n 1 vllm.log; sleep 5; done

2024-12-07 12:43:05.772122: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-07 12:43:08.278572: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-12-07 12:43:08.278572: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
INFO 12-07 12:43:17 api_server.py:194] Started engine process with PID 34843
INFO 12-07 12:43:17 api_server.py:194] Started engine process with PID 34843
2024-12-07 12:43:28.647433: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-12-07 12:43:28.647433: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
WARNING 12-07 12:43:38 arg_utils.py:1075] [DEPRECATED] Block manager v1 has been removed, and setting --use-v2-block-manager to True or 

Error: 400


In [25]:
import shutil

# Path to the directory
folder_path = "/content/llama3.2-fine-tuned"
output_zip = "/content/llama3.2-fine-tuned.zip"

# Compress the folder into a zip file
shutil.make_archive(folder_path, 'zip', folder_path)

# Provide a link to download the zip file
from google.colab import files
files.download(output_zip)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>